# Lecture 4 - Parsers

See below the code for an easy recognizer for the grammar 

```
A ::= id | B
B ::= int | (A)
```

In [8]:
type token = ID of string | INT of int | LBRA | RBRA | EOF

exception SyntaxError of string

let rec parseA (ts : token list) : token list = 
    match ts with 
    | ID x :: ts' -> ts'
    | _ -> parseB ts
    
and parseB (ts: token list) : token list = 
    match ts with
    | INT x :: ts' -> ts'
    | LBRA :: ts' -> (let ts'' = parseA ts'
                       in (match ts'' with 
                           | RBRA :: ts''' -> ts'''
                           | _ -> raise (SyntaxError "Expected RBRA.")
                           ))
    | _ -> raise (SyntaxError "Expected INT or LBRA") 

type token = ID of string | INT of int | LBRA | RBRA | EOF


exception SyntaxError of string


val parseA : token list -> token list = <fun>
val parseB : token list -> token list = <fun>


This is easily extended to a full parser which also yields a parse tree: 

In [9]:
type parsetree = A1 of token | A2 of parsetree | B1 of token | B2 of token * parsetree * token 

let rec parseA (ts : token list) : token list * parsetree = 
    match ts with 
    | ID x :: ts' -> (ts', A1 (ID x))
    | _ -> let (ts',ast) = parseB ts
           in (ts', A2 ast)
    
and parseB (ts: token list) : token list * parsetree = 
    match ts with
    | INT x :: ts' -> (ts', B1 (INT x))
    | LBRA :: ts' -> (let (ts'', ast) = parseA ts'
                       in (match ts'' with 
                           | RBRA :: ts''' -> (ts''', B2 (LBRA, ast, RBRA))
                           | _ -> raise (SyntaxError "Expected RPAREN.")
                           ))
    | _ -> raise (SyntaxError "Expected INT or LBRA") 

type parsetree =
    A1 of token
  | A2 of parsetree
  | B1 of token
  | B2 of token * parsetree * token


val parseA : token list -> token list * parsetree = <fun>
val parseB : token list -> token list * parsetree = <fun>
